In [2]:
import folium
import requests, bs4
import numpy as np
import pandas as pd
from lxml import html
import time
from tqdm import tqdm
from urllib.request import Request, urlopen
from urllib.parse import urlencode, quote_plus, unquote
from urllib import parse
from urllib.error import HTTPError
import json
pd.set_option('display.max_columns', 500)

In [3]:
#서비스 URL 주소
xmlUrl = 'http://openapi.epost.go.kr/postal/retrieveNewZipCdService/retrieveNewZipCdService/getNewZipCdList'
#공공데이터 포털에서 받은 API 인증키
API_Key = unquote('인증키')

In [4]:
def postnoadd(juso):
    queryParams = '?' + urlencode(    # get 방식으로 쿼리를 분리하기 위해 '?'를 넣은 것이다. 메타코드 아님.
        {
            quote_plus('serviceKey') : API_Key,
            # 필수 항목 1 : 인증키 (본인의 서비스키)
            quote_plus('srchwrd') : juso     
         }
    )
    response = requests.get(xmlUrl + queryParams).text.encode('utf-8')
    xmlobj = bs4.BeautifulSoup(response, 'lxml-xml')

    rows = xmlobj.findAll('newZipCdList')
    # rows # 디버깅용.

    rowList = []
    nameList = []
    columnList = []

    rowsLen = len(rows)
    for i in range(0, rowsLen):
        columns = rows[i].find_all()

        columnsLen = len(columns)
        for j in range(0, columnsLen):
            if i == 0:
                nameList.append(columns[j].name)
            eachColumn = columns[j].text
            columnList.append(eachColumn)
        rowList.append(columnList)
        columnList = []   

    result = pd.DataFrame(rowList, columns=nameList)
    return result['zipNo'][0]

In [5]:
postnoadd('서울시 중구 소공로70')

'04535'

In [6]:
Juso = pd.read_csv('POST_220725_8300.csv')

In [7]:
Juso

,주소표현형 일련번호,시도명,시군구명,법정읍면동명,도로명,건물본번,건물부번,기초구역번호,주소 기본형,주소표현형,Unnamed: 10
0,K068,대구광역시,동구,신서동,첨단로,53,0,41068.0,한국지능정보사회진흥원,한국정보화진흥원,NaN
1,K068002,대구광역시,동구,신서동,첨단로,53,0,41068.0,한국지능정보사회진흥원,정보화진흥원 본사,NaN
2,K068003,대구광역시,동구,신서동,첨단로,53,0,41068.0,한국지능정보사회진흥원,NIA 대구청사,NaN
3,K068004,대구광역시,동구,신서동,첨단로,53,0,41068.0,한국지능정보사회진흥원,한국지능정보사회진흥원,NaN
4,K068005,대구광역시,동구,신서동,첨단로,53,0,41068.0,한국지능정보사회진흥원,지능정보사회진흥원,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8293,NaN,경기도,의정부시,녹양동,진등로,21,0,NaN,녹양동주민센터,녹양동사무소,NaN
8294,NaN,경기도,의정부시,녹양동,진등로,21,0,NaN,녹양동주민센터,녹양동 작은도서관,NaN
8295,NaN,경기도,의정부시,의정부동,의정로,41,0,NaN,의정부정보도서관,의정부정보도서관,NaN
8296,NaN,경기도,의정부시,의정부동,의정로,41,0,NaN,의정부정보도서관,의정부시의회,NaN


In [10]:
for i in tqdm(range(3400,8298)):
    try:
        Juso.loc[i,'기초구역번호'] = postnoadd(Juso.loc[i,'시도명']+" "+Juso.loc[i,'시군구명']+" "+Juso.loc[i,'도로명']+" "+Juso.loc[i,'건물본번'])
    except KeyError:
        continue

100%|███████████████████████████████████████| 4898/4898 [10:41<00:00,  7.63it/s]


In [12]:
juso['기초구역번호']=juso['기초구역번호'].apply(str)

NameError: name 'juso' is not defined

In [11]:
Juso.to_csv('juso_8200csv')